## Title # Group Project Proposal:


### Introduction:
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question

### Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

### Methods:
Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

### Expected outcomes and significance:
1. What do you expect to find?
2. What impact could such findings have?
3. What future questions could this lead to?

In [1]:
library(tidyverse)
library(scales)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor




In [2]:
data <- read_csv("heart.numbers.csv")
data

Parsed with column specification:
cols(
  `PK` = col_character()
)

Warning message:
“6486 parsing failures.
row            col           expected actual                file
  2 PK delimiter or quote      'heart.numbers.csv'
  2 PK delimiter or quote     'heart.numbers.csv'
  2 PK delimiter or quote    'heart.numbers.csv'
  2 PK delimiter or quote    'heart.numbers.csv'
  2 PK delimiter or quote    'heart.numbers.csv'
... .............. .................. ...... ...................
See problems(...) for more details.
”
Warning message in FUN(X[[i]], ...):
“input string 1 is invalid in this locale”
Warning message in FUN(X[[i]], ...):
“input string 1 is invalid in this locale”
Warning message in FUN(X[[i]], ...):
“input string 1 is invalid in this locale”
Warning message in FUN(X[[i]], ...):
“input string 1 is invalid in this locale”
Warning message in FUN(X[[i]], ...):
“input string 1 is invalid in this locale”
Warning message in FUN(X[[i]], ...):
“input string 1 

PK
<chr>
""
en_CA:<82>5J P`


<8e>
<8c><8d> ĩ (ᱟ<8d>

 <88>`

